# Counting canceled appointments

In [1]:
import pandas as pd

In [2]:
# Sample data for testing
dataset = pd.DataFrame([
                       ["ala", pd.Timestamp('20130201 00:00:00'), 2, 1],\
                       ["ala", pd.Timestamp('20130201 00:00:00'), 2, 0],\
                       ["ola", pd.Timestamp('20130121 00:00:00'), 2, 0],\
                       ["ola", pd.Timestamp('20130312 00:00:00'), 2, 1],\
                       ["ala", pd.Timestamp('20130111 00:00:00'), 2, 1],\
                       ["ola", pd.Timestamp('20130901 00:00:00'), 2, 0],\
                       ["ola", pd.Timestamp('20130115 00:00:00'), 2, 1],\
                       ["ola", pd.Timestamp('20130801 00:00:00'), 2, 0],\
                       ["ala", pd.Timestamp('20130509 00:00:00'), 2, 1],\
                       ["ola", pd.Timestamp('20130211 00:00:00'), 2, 1]],\
                       columns=["PatientId","AppointmentDay","dummy","No-show"])

dataset

,PatientId,AppointmentDay,dummy,No-show
0,ala,2013-02-01,2,1
1,ala,2013-02-01,2,0
2,ola,2013-01-21,2,0
3,ola,2013-03-12,2,1
4,ala,2013-01-11,2,1
5,ola,2013-09-01,2,0
6,ola,2013-01-15,2,1
7,ola,2013-08-01,2,0
8,ala,2013-05-09,2,1
9,ola,2013-02-11,2,1


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientId       10 non-null     object        
 1   AppointmentDay  10 non-null     datetime64[ns]
 2   dummy           10 non-null     int64         
 3   No-show         10 non-null     int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 448.0+ bytes


### Counting canceled appointments for each patient 

In [4]:
# Counting canceled appointment for each patient (including current appointment)
dataset["sum_No-show"] = dataset.sort_values(["AppointmentDay"]).groupby("PatientId")["No-show"].cumsum().fillna(0)

dataset.sort_values(["PatientId","AppointmentDay"])

,PatientId,AppointmentDay,dummy,No-show,sum_No-show
4,ala,2013-01-11,2,1,1
0,ala,2013-02-01,2,1,2
1,ala,2013-02-01,2,0,2
8,ala,2013-05-09,2,1,3
6,ola,2013-01-15,2,1,1
2,ola,2013-01-21,2,0,1
9,ola,2013-02-11,2,1,2
3,ola,2013-03-12,2,1,3
7,ola,2013-08-01,2,0,3
5,ola,2013-09-01,2,0,3


<span style="color:blue">
Dla dni w których pacjent ma więcej niż jedną wizytę, powyższe rozwiązanie może zwrócić inne wyniki niż rozwiązanie Magdy. Wynika to z tego, że kolejność wizyt w tym zbiorze nie jest określona, podczas, gdy Magda zlicza je w kolejności indeksów. Generalnie jednak ta suma nie nadaje się, jako feature do modelu, bo zawiera nie tylko wizyty historyczne pacjenta, ale także bieżącą wizytę dla której realizujemy predykcję.<br><br>
Poniżej poprawne zliczanie, tj. bez uwzględniania bieżącej wizyty (shift powoduje przesunięcie o 1 rekord). Zliczanie realizuję w pętli po grupach (idxs to indeksy wszystkich rekordów dot. tego samego pacjenta), aby shift realizował przesunięcie w ramach rekordów tego samego pacjenta.
</span>

In [5]:
# Counting canceled appointment for each patient (without current appointment)
for idxs in dataset.sort_values("AppointmentDay").groupby("PatientId").groups.values():
    dataset.loc[idxs,"sum_No-show"] = dataset.loc[idxs, "No-show"].shift(1).cumsum().fillna(0)

dataset.sort_values(["PatientId","AppointmentDay"]) 

,PatientId,AppointmentDay,dummy,No-show,sum_No-show
4,ala,2013-01-11,2,1,0.0
0,ala,2013-02-01,2,1,1.0
1,ala,2013-02-01,2,0,2.0
8,ala,2013-05-09,2,1,2.0
6,ola,2013-01-15,2,1,0.0
2,ola,2013-01-21,2,0,1.0
9,ola,2013-02-11,2,1,1.0
3,ola,2013-03-12,2,1,2.0
7,ola,2013-08-01,2,0,3.0
5,ola,2013-09-01,2,0,3.0


### Counting recent canceled appointment for each patient (proposition):

Including the time window ('90D' = 90 days):

In [6]:
# Number of canceled visits for a given patient - in the last 90 days, without including the current visit
for idxs in dataset.sort_values("AppointmentDay").groupby("PatientId").groups.values():
    dataset.loc[idxs,"sum1_recent_No-show"] = dataset.loc[idxs].\
            rolling(window='90D', on="AppointmentDay", min_periods=1, center=False, closed="left")["No-show"].sum().fillna(0)

dataset.sort_values(["PatientId","AppointmentDay"])

,PatientId,AppointmentDay,dummy,No-show,sum_No-show,sum1_recent_No-show
4,ala,2013-01-11,2,1,0.0,0.0
0,ala,2013-02-01,2,1,1.0,1.0
1,ala,2013-02-01,2,0,2.0,2.0
8,ala,2013-05-09,2,1,2.0,0.0
6,ola,2013-01-15,2,1,0.0,0.0
2,ola,2013-01-21,2,0,1.0,1.0
9,ola,2013-02-11,2,1,1.0,1.0
3,ola,2013-03-12,2,1,2.0,2.0
7,ola,2013-08-01,2,0,3.0,0.0
5,ola,2013-09-01,2,0,3.0,0.0


Alternatively, without the time window, but including only the last X visits:

In [7]:
# Number of canceled visits for a given patient - within the last 5 visits, without including the current one
dataset["sum2_recent_No-shows"] = dataset.sort_values("AppointmentDay").groupby("PatientId")["No-show"].\
        transform(lambda x: x.rolling(window=5, min_periods=1, center=False, closed="left").sum().fillna(0))

dataset.sort_values(["PatientId","AppointmentDay"])

,PatientId,AppointmentDay,dummy,No-show,sum_No-show,sum1_recent_No-show,sum2_recent_No-shows
4,ala,2013-01-11,2,1,0.0,0.0,0
0,ala,2013-02-01,2,1,1.0,1.0,1
1,ala,2013-02-01,2,0,2.0,2.0,2
8,ala,2013-05-09,2,1,2.0,0.0,2
6,ola,2013-01-15,2,1,0.0,0.0,0
2,ola,2013-01-21,2,0,1.0,1.0,1
9,ola,2013-02-11,2,1,1.0,1.0,1
3,ola,2013-03-12,2,1,2.0,2.0,2
7,ola,2013-08-01,2,0,3.0,0.0,3
5,ola,2013-09-01,2,0,3.0,0.0,3


### Number of canceled visits since a day with a last successful visit

In [8]:
for idxs in dataset.sort_values("AppointmentDay").groupby("PatientId").groups.values():          
    # Last successful visit and number of canceled visits since a last successful visit   
    last_show = pd.Timestamp(0)
    sumRecentNoShows = 0
    for i in idxs:
        dataset.loc[i,"_lastShow"] = last_show
        dataset.loc[i,"_sumRecentNoShows"] = sumRecentNoShows
        if dataset.loc[i,"No-show"] == 0:
            last_show = dataset.loc[i,"AppointmentDay"]
            sumRecentNoShows = 0
        else:
            sumRecentNoShows += 1   
            
dataset.sort_values(["PatientId","AppointmentDay"])

,PatientId,AppointmentDay,dummy,No-show,sum_No-show,sum1_recent_No-show,sum2_recent_No-shows,_lastShow,_sumRecentNoShows
4,ala,2013-01-11,2,1,0.0,0.0,0,1970-01-01,0.0
0,ala,2013-02-01,2,1,1.0,1.0,1,1970-01-01,1.0
1,ala,2013-02-01,2,0,2.0,2.0,2,1970-01-01,2.0
8,ala,2013-05-09,2,1,2.0,0.0,2,2013-02-01,0.0
6,ola,2013-01-15,2,1,0.0,0.0,0,1970-01-01,0.0
2,ola,2013-01-21,2,0,1.0,1.0,1,1970-01-01,1.0
9,ola,2013-02-11,2,1,1.0,1.0,1,2013-01-21,0.0
3,ola,2013-03-12,2,1,2.0,2.0,2,2013-01-21,1.0
7,ola,2013-08-01,2,0,3.0,0.0,3,2013-01-21,2.0
5,ola,2013-09-01,2,0,3.0,0.0,3,2013-08-01,0.0


In [9]:
dataset

,PatientId,AppointmentDay,dummy,No-show,sum_No-show,sum1_recent_No-show,sum2_recent_No-shows,_lastShow,_sumRecentNoShows
0,ala,2013-02-01,2,1,1.0,1.0,1,1970-01-01,1.0
1,ala,2013-02-01,2,0,2.0,2.0,2,1970-01-01,2.0
2,ola,2013-01-21,2,0,1.0,1.0,1,1970-01-01,1.0
3,ola,2013-03-12,2,1,2.0,2.0,2,2013-01-21,1.0
4,ala,2013-01-11,2,1,0.0,0.0,0,1970-01-01,0.0
5,ola,2013-09-01,2,0,3.0,0.0,3,2013-08-01,0.0
6,ola,2013-01-15,2,1,0.0,0.0,0,1970-01-01,0.0
7,ola,2013-08-01,2,0,3.0,0.0,3,2013-01-21,2.0
8,ala,2013-05-09,2,1,2.0,0.0,2,2013-02-01,0.0
9,ola,2013-02-11,2,1,1.0,1.0,1,2013-01-21,0.0
